In [4]:
env = gym.make("Taxi-v2").env
env.render()

+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+



In [9]:
env.reset()
env.render()

print("Action Space {}".format(env.action_space))
print("State Space {}".format(env.observation_space))

+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+

Action Space Discrete(6)
State Space Discrete(500)


In [21]:
state = env.encode(3, 1, 2, 0) # (taxi row, taxi column, passenger index, destination index)
print("State:", state)

env.s = state
env.render(mode='human')

State: 328
+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (Dropoff)


In [11]:
env.P[328]

{0: [(1.0, 428, -1, False)],
 1: [(1.0, 228, -1, False)],
 2: [(1.0, 348, -1, False)],
 3: [(1.0, 328, -1, False)],
 4: [(1.0, 328, -10, False)],
 5: [(1.0, 328, -10, False)]}

In [22]:
###In This example, we show random movement
###And clearly demonstrate how ineffective it is

env.s = 328

epochs = 0
penalties, reward = 0, 0

frames = [] #animation
done = False

while not done:
    action = env.action_space.sample()
    state, reward, done, info = env.step(action)
    
    if reward == -10:
        penalties += 1
    #put each fram into dict for animation
    frames.append({
        'frame': env.render(mode='ansi'),
        'state': state,
        'action': action,
        'reward': reward
    })
    
    epochs += 1

print("Timesteps taken: {}".format(epochs))
print("Penalties Incurred: {}".format(penalties))
    

Timesteps taken: 502
Penalties Incurred: 159


In [23]:
from IPython.display import clear_output
from time import sleep

def print_frames(frames):
    for i, frame in enumerate(frames):
        clear_output(wait=True)
        print(frame['frame'])
        print(f"Timestep: {i + 1}")
        print(f"State: {frame['state']}")
        print(f"Action: {frame['action']}")      
        print(f"Reward: {frame['reward']}")
        sleep(.1)

print_frames(frames)

+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (East)

Timestep: 229
State: 448
Action: 2
Reward: -1


KeyboardInterrupt: 

In [17]:
%%time
import numpy as np

q_table = np.zeros([env.observation_space.n, env.action_space.n])

"""Training the agent"""

import random
from IPython.display import clear_output

#Hyperparameters
alpha = .1
gamma = .6
epsilon = .1

#For plotting metrics
all_epochs = []
all_penalties = []

for i in range(1, 100001):
    state = env.reset()
    
    epochs, penalties, reward, = 0,0,0
    done = False
    
    while not done:
        if random.uniform(0,1) < epsilon:
            action = env.action_space.sample() #explore!
        else:
            action = np.argmax(q_table[state])
            
        next_state, reward, done, info = env.step(action)
        
        old_value = q_table[state, action]
        next_max = np.max(q_table[next_state])
        
        new_value = (1-alpha) * old_value + alpha * (reward + gamma * next_max)
        q_table[state, action] = new_value
        
        if reward == -10:
            penalties += 1
            
        state = next_state
        epochs += 1
        
    if i % 100 == 0:
        clear_output(wait=True)
        print(f"Episode: {i}")
        
print("Training finished.\n")
 
    

Episode: 100000
Training finished.

CPU times: user 32 s, sys: 1.41 s, total: 33.4 s
Wall time: 31.2 s


In [24]:
"""Evaluate agent's performance after Q-Learning"""


total_epochs, total_penalties = 0,0
episodes = 20

frames = []

for _ in range(episodes):
    state = env.reset()
    epochs, penalties, reward = 0,0,0
    
    done = False
    
    while not done: 
        action = np.argmax(q_table[state])
        state, reward, done, info = env.step(action)
        
        frames.append({
        'frame': env.render(mode='ansi'),
        'state': state,
        'action': action,
        'reward': reward
        })
        
        if reward == -10:
            penalties += 1
            
        epochs +=1
    
    total_penalties += penalties
    total_epochs += epochs
    
    
print_frames(frames)
print(f"Results after {episodes} episodes:")
print(f"Average timesteps per episode: {total_epochs / episodes}")
print(f"Average penalties per episode: {total_penalties / episodes}")
        


+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (Dropoff)

Timestep: 257
State: 0
Action: 5
Reward: 20
Results after 20 episodes:
Average timesteps per episode: 12.85
Average penalties per episode: 0.0
